In [ ]:
# ugly fix for the shadow of the map widgets
from IPython.display import display, HTML

display(HTML("<style>.leaflet-widgetcontrol {box-shadow: none}</style>"))

In [ ]:
from sepal_ui import sepalwidgets as sw
from component import widget as cw
from component.message import cm

In [ ]:
# create an appBar
rp_appBar = sw.AppBar(cm.app.title)

In [ ]:
# add tiles
%run 'aoi_ui.ipynb'
%run 'questionnaire_ui.ipynb'
%run 'compute_ui.ipynb'
%run 'about_ui.ipynb'

rp_content = [
    rp_aoi,
    rp_questionnaire,
    rp_validation,
    rp_map_tile,
    rp_regions,
    rp_theme,
    rp_about,
    rp_disclaimer,
]

In [ ]:
# create a drawer
items = [
    sw.DrawerItem(cm.app.drawer.about, "mdi-help-circle", card="about_tile"),
    sw.DrawerItem(cm.app.drawer.aoi, "mdi-map-marker-check", card="aoi_tile"),
    sw.DrawerItem(
        cm.app.drawer.question, "mdi-file-question", card="questionnaire_widget"
    ),
    sw.DrawerItem(cm.app.drawer.recipe, "mdi-cogs", card="compute_widget"),
    sw.DrawerItem(cm.app.drawer.map, "mdi-map", card="map_widget"),
    sw.DrawerItem(
        cm.app.drawer.dashboard, "mdi-view-dashboard", card="dashboard_widget"
    ),
    # sw.DrawerItem(
    #    cm.app.drawer.feedback,
    #    "mdi-file-document-edit-outline",
    #    href="https://forms.gle/EpJkL3gptFfbUKCm9",
    # ),
]

code_link = "https://github.com/12rambau/restoration_planning_module"
wiki_link = "https://sepal-ui.readthedocs.io/en/latest/modules/dwn/seplan.html"
issue_link = "https://github.com/12rambau/restoration_planning_module/issues/new"

rp_drawer = sw.NavDrawer(items, code=code_link, wiki=wiki_link, issue=issue_link)

In [ ]:
# build the app
import ipyvuetify as v

alert_msg = "The se.plan application is still under development. We encourage the users to continue explore the tool functionalities and report any issues using the « bug report » link in the left panel."

rp_app = (
    cw.CustomApp(tiles=rp_content, appBar=rp_appBar, navDrawer=rp_drawer)
    .show_tile("about_tile")
    .add_alert(alert_msg, type="warning")
)

In [ ]:
# display the app
rp_app